In [1]:
import geopandas as gpd
import folium
from pathlib import Path

In [2]:
project_dir = Path(r"d:\projecten\D2203.HDSR-modelvergelijking")

In [3]:
peilgebieden_gdf = gpd.read_file(
    project_dir.joinpath(r"03.Bronbestanden\Watersysteem\Peilgebieden.geojson")
)

meetpunten_de_tol_gdf = gpd.read_file(
    project_dir.joinpath(r"04.Modelvergelijking\de_tol\debiet_meetpunten.gpkg")
)

meetpunten_ameronger_wetering_gdf = gpd.read_file(
    project_dir.joinpath(r"04.Modelvergelijking\amerongerwetering\debiet_meetpunten.gpkg")
)

# De Tol

In [4]:
# masks waarmee we alleen selecteren wat we nodig hebben
peilgebieden_mask = peilgebieden_gdf["ONDAFVOGEB"] == "AFVGEB0012"
meetpunten_mask = meetpunten_de_tol_gdf["CODE"].isin(["ST2037", "ST2036", "ST0136", "ST2909", "ST2902", "G2901"])


# samenstellen van peilgebieden
waterbalans_dict = dict(
    ST2902= ["PG0336", "PG0403"],
    ST2909= ["PG0404"],
    ST2036 = ["PG0397"],
    G2901 = ["PG0401","PG0402"], 
    ST0136 = ["PG0394","PG1767","PG2537","PG2182","PG1768"],
    ST2037 = ["PG0834", "PG2484", "PG0398", "PG0399", "PG2553", "PG0400", "PG2555"]
)
waterbalans_gdf = gpd.GeoDataFrame(
    {"code": list(waterbalans_dict.keys()),
     "geometry": [peilgebieden_gdf[
         peilgebieden_gdf["CODE"].isin(i)
     ].geometry.unary_union for i in waterbalans_dict.values()]},
    crs="epsg:4326"
)
waterbalans_gdf.to_crs("epsg:28992")

# we plotten een kaartje om ons resultaat te beoordelen
m = waterbalans_gdf.explore(name="waterbalans",
                           style_kwds={"fill":True, "color":"green"})
peilgebieden_gdf.loc[peilgebieden_mask].explore(
    m=m,
    name="peilgebieden",
    style_kwds={"fill":False, "color":"blue"})
meetpunten_de_tol_gdf[meetpunten_mask].explore(
    m=m,
    color="red",
    name="meetpunten")

folium.LayerControl().add_to(m)

waterbalans_gdf.to_file(
    project_dir.joinpath(r"04.Modelvergelijking\de_tol\waterbalans.gpkg"),
    driver="GPKG")

m